In [16]:
import os, json, csv
from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

___

## Data Processing (70%)

### Step 1: Reading the 'csv' File


In [7]:
def read_csv(file_path):
    data = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)
    return data

csv_file_path = os.path.join(os.getcwd(), "acw_user_data.csv")
csv_data = read_csv(csv_file_path)

### Step 2: Structuring the Data

In [8]:
def nest_data(row):
    dependants = row.get("Dependants", "").strip()
    if not dependants.isdigit():
        dependants = None

    retired = row.get("Retired", "").strip().lower() == "true"

    return {
        "first_name": row.get("First Name"),
        "second_name": row.get("Last Name"),
        "age": int(row.get("Age (Years)", 0)),
        "sex": row.get("Sex"),
        "retired": retired,
        "marital_status": row.get("Marital Status"),
        "dependants": dependants,
        "salary": int(row.get("Yearly Salary (Dollar)", 0)),
        "pension": int(row.get("Yearly Pension (Dollar)", 0)),
        "company": row.get("Employer Company"),
        "commute_distance": float(row.get("Distance Commuted to Work (Km)", 1.0)),
        "Vehicle": {
            "make": row.get("Vehicle Make"),
            "model": row.get("Vehicle Model"),
            "year": row.get("Vehicle Year"),
            "category": row.get("Vehicle Type")
        },
        "Credit Card": {
            "start_date": row.get("Credit Card Start Date"),
            "end_date": row.get("Credit Card Expiry Date"),
            "number": row.get("Credit Card Number"),
            "ccv": row.get("Credit Card CVV"),
            "iban": row.get("Bank IBAN")
        },
        "Address": {
            "street": row.get("Address Street"),
            "city": row.get("Address City"),
            "postcode": row.get("Address Postcode")
        }
    }

### Step 3: Tracking Problematic Rows

In [9]:
def process_data(data):
    _processed_data = []
    problematic_rows = []
    for idx, row in enumerate(data):
        nested_row = nest_data(row)
        if nested_row["dependants"] is None:
            problematic_rows.append(idx)
            nested_row["dependants"] = 0
        _processed_data.append(nested_row)
    print("Problematic rows for dependants:", problematic_rows)
    return _processed_data

processed_data = process_data(csv_data)

Problematic rows for dependants: [21, 109, 179, 205, 270, 272, 274, 358, 460, 468, 579, 636, 679, 725, 822, 865, 917, 931, 983]


### Step 4: Saving Processed Data to JSON

In [17]:
def save_to_json(data, file_name="processed.json"):
    with open(file_name, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

save_to_json(processed_data, "json_data/processed.json")

### Step 5: Separation by Retirement Status

In [18]:
def separate_by_retirement(data):
    retired = [entry for entry in data if entry["retired"] is True]
    employed = [entry for entry in data if entry["retired"] is False]
    with open("json_data/retired.json", "w", encoding="utf-8") as file:
        json.dump(retired, file, ensure_ascii=False, indent=4)
    with open("json_data/employed.json", "w", encoding="utf-8") as file:
        json.dump(employed, file, ensure_ascii=False, indent=4)

separate_by_retirement(processed_data)

### Step 6: Flagging Outdated Credit Cards

In [19]:
def flag_outdated_credit_cards(data):
    outdated_cards = []

    for entry in data:
        start_date = entry["Credit Card"]["start_date"]
        end_date = entry["Credit Card"]["end_date"]

        if start_date and end_date:
            start_year = int(start_date.split("/")[-1])
            end_year = int(end_date.split("/")[-1])

            start_year += 2000 if start_year < 100 else 0
            end_year += 2000 if end_year < 100 else 0

            if (end_year - start_year) > 10:
                outdated_cards.append(entry)

    with open("json_data/remove_ccard.json", "w", encoding="utf-8") as file:
        json.dump(outdated_cards, file, ensure_ascii=False, indent=4)

flag_outdated_credit_cards(processed_data)

### Step 7: Salary-Commute Calculation and Sorting

In [20]:
def calculate_salary_commute(data):
    for entry in data:
        commute_distance = entry["commute_distance"]
        if commute_distance > 1:
            entry["SalaryCommute"] = entry["salary"] / commute_distance
        else:
            entry["SalaryCommute"] = entry["salary"]

    sorted_data = sorted(data, key=lambda x: x["SalaryCommute"], reverse=True)

    with open("json_data/commute.json", "w", encoding="utf-8") as file:
        json.dump(sorted_data, file, ensure_ascii=False, indent=4)

calculate_salary_commute(processed_data)

___

## Data Visualisation (20%)

Your client wishes to understand the data they have on their customers a bit more by use of visualisations. With use of Pandas and Seaborn read in the original CSV file provided with the assignment.

##### 1. Obtain the Data Series for Salary, and Age, and calculate the following:
   - Mean Salary 
   - Median Age

##### 2. Perform uni variate plots of the following data attributes:
   - Age, calculating how many bins would be required for a bin_width of 5. 
   - Dependents, fixing data errors with seaborn itself. 
   - Age (of default bins), conditioned on Marital Status

##### 3. Perform multivariate plots with the following data attributes:
   - Commuted distance against salary.
   - Age against Salary
   - Age against Salary conditioned by Dependants

##### 4. Your client would like the ability to save the plots which you have produced. Provide a Notebook cell which can do this.